### Libs & Data

In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import string
import random
import unicodedata
from string import punctuation
from string import digits
from nltk.stem import WordNetLemmatizer
import joblib

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# from sklearn.manifold import TSNE
# from sklearn.cluster import KMeans
# from sklearn.cluster import DBSCAN
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

import torch
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

import spacy
from spacy.matcher import Matcher

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load large spacy model 
nlp = spacy.load('en_core_web_lg')

In [5]:
df_sum = pd.read_csv("data/data_sum.csv")
df_sum.head()

,title,author,time,description,body,section,summarized_body
0,A snapshot of multiculturalism in South Korea,Lee Sun-young,2018-01-01 17:07:00,With birthrates persistently low and the senio...,With birthrates persistently low and the senio...,Social affairs,"As of 2016, more than 2 million foreign nation..."
1,[Weekender] Korea’s dynamic 2017,Choi He-suk,2018-01-01 13:22:00,From North Korea’s nuclear weapons program nea...,From North Korea’s nuclear weapons program nea...,Social affairs,The corruption scandal that broke out in 2016 ...
2,People's Party members support Ahn's push for ...,Yonhap,2017-12-31 16:18:00,The leader of the center-left People's Party g...,The leader of the center-left People's Party g...,Politics,"Ahn Cheol-soo, leader of the center-left Peopl..."
3,[Newsmaker] Panamanian vessel probed over susp...,Yonhap,2017-12-31 14:55:00,PYEONGTAEK -- South Korea has seized and insp...,PYEONGTAEK -- South Korea has seized and insp...,North Korea,"The 5,100-ton KOTI is being held in the wester..."
4,Hong Kong ship crew questioned in S. Korea for...,AFP,2017-12-30 15:44:00,The crew of a Hong Kong-registered ship have b...,The crew of a Hong Kong-registered ship have b...,North Korea,The crew of a Hong Kong-registered ship have b...


In [6]:
df_nk = pd.read_csv("data/df_nk_labeled")
df_nk.head()

,Unnamed: 0,title,author,time,description,body,section,summarized_body,agg_title_body,label
0,3,[Newsmaker] Panamanian vessel probed over susp...,Yonhap,2017-12-31 14:55:00,PYEONGTAEK -- South Korea has seized and insp...,PYEONGTAEK -- South Korea has seized and insp...,North Korea,"The 5,100-ton KOTI is being held in the wester...",[Newsmaker] Panamanian vessel probed over susp...,0
1,7,Secret Sauce? Kim Jong-un applies science to k...,AP,2017-12-30 12:10:00,Kim Jong Un wants to turn the art of kimchi-ma...,Kim Jong Un wants to turn the art of kimchi-ma...,North Korea,"Ryugyong Kimchi Factory produces 4,200 tons of...",Secret Sauce? Kim Jong-un applies science to k...,1
2,8,N. Korea says there will be no change to its n...,Yonhap,2017-12-30 10:31:00,North Korea will continue to enhance its nucle...,North Korea will continue to enhance its nucle...,North Korea,North Korea will continue to enhance its nucle...,N. Korea says there will be no change to its n...,2
3,14,"Top diplomats of S. Korea, US reassure peacefu...",Yonhap,2017-12-29 16:31:00,The top diplomats of South Korea and the Unite...,The top diplomats of South Korea and the Unite...,North Korea,Top diplomats of South Korea and the U.S. held...,"Top diplomats of S. Korea, US reassure peacefu...",3
4,19,Businessmen call for probe into shutdown of fa...,Yonhap,2017-12-29 15:04:00,A private task force on Friday pressed the gov...,A private task force on Friday pressed the gov...,North Korea,South Korea pulled the plug on the factory par...,Businessmen call for probe into shutdown of fa...,4


In [7]:
top_nk = pd.read_csv("data/top_nk.csv")
top_nk

,summary,label,freq
0,North Korea appears to be advancing its submar...,100,13
1,Group of 13 North Korean defectors who worked ...,1265,13
2,Around 200 propaganda leaflets from the North ...,547,10
3,254 people of 90 families from the South met w...,2216,9
4,84 percent of Americans think President Donald...,203,8
5,UN nuclear watchdog should revise its view on ...,778,7
6,U.N. Security Council adopted Resolution 2270 ...,1403,7
7,IOC President Thomas Bach is seeking to visit ...,13,7
8,North Korea has featured a flight crew of its ...,751,6
9,Shallow 3.5-magnitude earthquake hits North Ko...,342,6


In [8]:
df_politics = pd.read_csv("data/df_politics_labeled")
df_politics.head()

,Unnamed: 0,title,author,time,description,body,section,summarized_body,agg_title_body,label
0,21,Special pardons aimed at helping ordinary peop...,Yonhap,2017-12-29 11:39:00,"The latest pardon extended to more than 6,000 ...","The latest pardon extended to more than 6,000 ...",Politics,"The latest pardon extended to more than 6,000 ...",Special pardons aimed at helping ordinary peop...,0
1,34,[News Focus] Is multiparty system viable in Ko...,Jo He-rim,2017-12-28 16:29:00,A four-day vote of confidence in the People’s ...,A four-day vote of confidence in the People’s ...,Politics,A four-day vote of confidence in the People’s ...,[News Focus] Is multiparty system viable in Ko...,1
2,37,Political parties call on government to resolv...,Jo He-rim,2017-12-28 16:13:00,Political parties on Thursday expressed anger ...,Political parties on Thursday expressed anger ...,Politics,Political parties on Thursday expressed anger ...,Political parties call on government to resolv...,2
3,84,Korea's ODA project tainted by corruption scan...,Yonhap,2017-12-26 16:02:00,Impeached President Park Geun-hye's close frie...,Impeached President Park Geun-hye's close frie...,Politics,The probe looked into allegations that the pri...,Korea's ODA project tainted by corruption scan...,3
4,174,Ex-President Park summoned for questioning Friday,Ock Hyun-ju,2017-12-20 17:51:00,The prosecution summoned former President Park...,The prosecution summoned former President Park...,Politics,The prosecution summoned former President Park...,Ex-President Park summoned for questioning Fri...,4


In [9]:
top_politics = pd.read_csv("data/top_politics.csv")
top_politics

,summary,label,freq
0,Moon Jae-in of the liberal Democratic Party ga...,360,11
1,Former UN Secretary-General Ban Ki-moon’s retu...,642,9
2,President Park Geun-hye on Friday granted spec...,1212,9
3,President Park Geun-hye vetoed the controversi...,1329,8
4,67.8 percent of respondents said they approved...,37,8
5,South Korean President Park Geun-hye returned ...,1939,8
6,Eight former renegade lawmakers returned to th...,28,7
7,Seoul court on Tuesday held the first preparat...,525,7
8,President Park Geun-hye arrived in Seoul on We...,1373,7
9,The Federation of Korean Trade Unions said Tue...,1504,7


### Merge Sentences (optional)

In [14]:
# Returns merged sentences given df and label
def merge_sents(df=df_nk, label=100):
    return df[df.label == label]['agg_title_body']

In [16]:
sents_nk = merge_sents(df_nk, 100).tolist()
sents_nk

['Satellite imagery shows progress in N. Korea\'s SLBM program. North Korea appears to be advancing its submarine-launched ballistic missile (SLBM) program, with indications a second testing platform is being readied for deployment, a U.S. website showed Friday.38 North, which specializes in North Korea analysis, said commercial satellite imagery from Nov. 11, 16 and 24 show the country\'s second platform for launching missiles underwater outside of submarines has been moved.\xa0Captured satellite image from 38 North (Yonhap)The platform, known as a submersible ballistic missile test stand barge, "is being prepared to enter service, a strong indicator that Pyongyang is advancing its SLBM program," the website said, citing analysis by expert Joseph S. Bermudez Jr.On Nov. 11, the barge was seen positioned on a construction way at the Nampo Navy Shipyard, with a floating dry-dock at the end of the construction way. Five days later the barge was seen aboard the floating dry-dock and towed 

In [17]:
sents_politics = merge_sents(df_politics, 1939).tolist()
sents_politics

['Park returns home after Mideast swing. South Korean President Park Geun-hye returned home Monday after a four-nation Middle East swing meant to further expand economic ties beyond oil and construction.The trip to four Gulf States -- Kuwait, Saudi Arabia, the United Arab Emirates and Qatar -- came as oil-rich Mideast countries are pushing to diversify their economic portfolios ahead of the inevitable advent of a post-oil era.South Korea believes that such attempts could present new business opportunities for its companies, especially in such fields as health care and information and communications technology.South Korea views the potential business opportunities as the "second Middle East boom" following the first one in the 1970s when many Korean workers sent home their hard-earned cash from sweating at construction sites in the region.The Middle Eastern countries "are offering us another chance with the second Middle East boom," Park said Sunday in a meeting with more than 100 South

### Summarize

In [35]:
df_nk[df_nk.label==100][['time', 'title', 'summarized_body']]

,time,title,summarized_body
116,2017-12-02 09:35:00,Satellite imagery shows progress in N. Korea's...,North Korea appears to be advancing its submar...
1197,2017-05-05 09:04:00,New activity seen at test stand of N. Korea's ...,Recent satellite imagery indicates new activit...
1715,2016-08-25 11:27:00,N. Korea developing larger-class submarine for...,North Korea is building a new bigger class of ...
1990,2016-05-04 11:22:00,Satellite imagery shows N. Korea actively purs...,Recent satellite imagery shows North Korea is ...
2029,2016-04-25 16:26:00,"N. Korea aims to build new 3,000-ton sub armed...",North Korea may be in the process of developin...
2042,2016-04-23 20:18:00,N.K. apparently fired ballistic missile from s...,The Joint Chiefs of Staff said North Korea fir...
2168,2016-03-18 09:34:00,N. Korea continues SLBM development: 38 North,U.S. website 38 North says North Korea appears...
2477,2016-01-13 09:38:00,N. Korea believed to have conducted latest SLB...,The December test is the North's third known S...
2502,2016-01-06 12:12:00,North Korea test-fired SLBM last month: South ...,North Korea conducted another ejection test of...
2505,2016-01-06 09:07:00,North Korea successfully conducts SLBM test la...,The ejection test of a KN-11 missile from a su...


In [40]:
df_nk[df_nk.label==100][['time', 'title', 'summarized_body']].iloc[::-1].values.tolist()

[['2015-05-13 09:08:00',
  "N. Korea's SLBM test conducted from barge, not from submarine: U.S. expert",
  'North Korea appears to have "photoshopped" images of the test to exaggerate progress, expert says. Joseph Bermudez says satellite imagery shows a submarine docked next to a barge. The missile used in the test is believed to be much shorter than the North\'s Musudan or Nodong.'],
 ['2015-10-16 09:33:00',
  'Pyongyang preparing to test launch tube used in SLBM: 38 North',
  'North Korea appears to be preparing to test a vertical launch tube used in submarine-launched ballistic missiles. Satellite imagery taken on Sept. 21 showed that a launch superstructure has been erected at the east coast shipyard. Such a structure is erected to conduct tests of the stabilization and fire-control systems and for ejection, or "pop up," tests.'],
 ['2015-12-01 09:17:00',
  "Pentagon declines comment on N.K.'s SLBM test",
  'Yonhap News Agency reported that the North fired a KN-11 missile from a su

In [36]:
df_politics[df_politics.label==1939][['time', 'title', 'summarized_body']]

,time,title,summarized_body
2312,2015-03-09 09:16:00,Park returns home after Mideast swing,South Korean President Park Geun-hye returned ...
2313,2015-03-08 18:35:00,"Korea, Qatar to bolster economic partnership",Leaders of South Korea and Qatar agreed to exp...
2316,2015-03-08 15:19:00,Park to hold 2nd summit with Qatari emir in fo...,Park Geun-hye to meet with Qatar's emir to dis...
2317,2015-03-07 15:13:00,Park to meet with S. Korean businessmen in Qatar,President Park Geun-hye plans to meet with Sou...
2331,2015-03-02 20:40:00,"Korea, Kuwait agree to diversify business rela...",President Park Geun-hye and the emir of Kuwait...
2333,2015-03-02 09:15:00,Park seeks to generate ‘second Middle East boom’,President Park Geun-hye says she will seek new...
2334,2015-03-01 18:47:00,President Park begins trip to Middle East,President Park Geun-hye arrives in Kuwait for ...
2340,2015-02-26 21:15:00,Park seeks to upgrade relations in Gulf,President Park Geun-hye is set to embark on he...


In [39]:
df_politics[df_politics.label==1939][['time', 'title', 'summarized_body']].iloc[::-1].values.tolist()

[['2015-02-26 21:15:00',
  'Park seeks to upgrade relations in Gulf',
  'President Park Geun-hye is set to embark on her nine-day trip on Sunday. She plans to visit Kuwait, Saudi Arabia, the United Arab Emirates and Qatar. This year marks the 50th year since South Korean builders entered foreign markets. Park urged officials to expand economic cooperation with the region.'],
 ['2015-03-01 18:47:00',
  'President Park begins trip to Middle East',
  'President Park Geun-hye arrives in Kuwait for first leg of four-nation trip to Middle East. Park is set to hold summits with leaders of four nations to expand cooperation. 116 representatives of public and private firms have joined the presidential trip. Business forums will be held in all the four Gulf nations to help South Korean firms.'],
 ['2015-03-02 09:15:00',
  'Park seeks to generate ‘second Middle East boom’',
  'President Park Geun-hye says she will seek new business opportunities for South Korean companies in the Middle East. Her 

### NER

In [44]:
### Lemmatization tool
stemmer = WordNetLemmatizer()
### Change similar words to the same word
UN_WORD = "The United Nations"
US_WORD = "The United States"
NK_WORD = "North Korea"
SK_WORD = "South Korea"
PARK_WORD = "Park Geun-hye"

similar_words = {
    # Change to "The United States"
    "U.S.": US_WORD,
    "US": US_WORD,
    "USA": US_WORD,
    "United States": US_WORD,
    "United States'": US_WORD,
    "The United States'": US_WORD,
    
    # Change to "North Korea"
    "NK": NK_WORD,
    "NK's": NK_WORD,
    "N. Korea": NK_WORD,
    "N. Korea's": NK_WORD,
    "North Korea's": NK_WORD,
    
    # Change to "South Korea"
    "SK": SK_WORD,
    "SK's": SK_WORD,
    "S. Korea": SK_WORD,
    "S. Korea's": SK_WORD,
    "South Korea's": SK_WORD,
    
    # Change to "The United Nations"
    "United Nations": UN_WORD,
    "United Nations'": UN_WORD,
    "The United Nations'": UN_WORD,
    "UN": UN_WORD,
    
    # Change to "Park Geun-hye"
    "President Park Geun-hye's": PARK_WORD,
    "President Park Geun-Hye's": PARK_WORD,
    "President Park Geun Hye's": PARK_WORD,
    "President Park Geun-hye": PARK_WORD,
    "President Park Geun-Hye": PARK_WORD,
    "President Park Geun Hye": PARK_WORD,
    "President Park's": PARK_WORD,
    "President Park": PARK_WORD,
    "Park Geun-hye's": PARK_WORD,
    "Park Geun-Hye's": PARK_WORD,
    "Park Geun Hye's": PARK_WORD,
    "Park Geun-Hye": PARK_WORD,
    "Park Geun Hye": PARK_WORD,
}

### Transform function
def transform_to_similar_sentence(s: str):
    new_str = s
    for key,value in similar_words.items():
        new_str = re.sub(key, value, new_str)
    return new_str

### Preprocess function for grouping similar topic
def preprocess_manual(s: str):
    # Change similar words to the same word
    new_str = transform_to_similar_sentence(s)
    # Remove punctuation
    new_str = ''.join(ch if ch not in set(punctuation) else " " for ch in new_str)
    # Remove all single characters
    new_str = re.sub(r'\W', ' ', new_str)
    new_str = re.sub(r'\s+[a-zA-Z]\s+', ' ', new_str)
    new_str = re.sub(r'\^[a-zA-Z]\s+', ' ', new_str) 
    # Substituting multiple spaces with single space
    new_str = re.sub(r'\s+', ' ', new_str, flags=re.I)
    # Removing prefixed 'b' - when data is in bytes format
    new_str = re.sub(r'^b\s+', '', new_str)
    # Removing all numbers
    new_str = new_str.translate(str.maketrans('', '', digits))
    # Converting to Lowercase
    new_str = new_str.lower()
    # Lemmatization and remove stopwords
    new_str = new_str.split()
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = [stemmer.lemmatize(word) for word in new_str if word not in stopwords]
    new_str = ' '.join(tokens)
    
    return new_str

In [86]:
# Extract entities from sentences
def extract_entity(sents=sents_nk, label=100, df=df_nk):
    print("[ Detailed Information (per event) ]\n")
    
    ent_per = []
    ent_org = []
    ent_loc = []
    
    event = df[df.label == label]['title'].iloc[::-1].values.tolist()
    
    for i in reversed(range(len(sents))):
        doc = nlp(sents[i])
        for ent in doc.ents:
            word = ent.text.title()
            if word in ent_per or word in ent_org or word in ent_loc:
                continue
            if ent.label_ == 'PERSON':
                ent_per.append(word)
            elif ent.label_ == 'ORG':
                ent_org.append(word)
            elif ent.label_ in ['GPE', 'LOC']:
                ent_loc.append(word)
        
        print("Event: ", event[i])
        print("- Person: ", ", ".join([i for i in ent_per]))
        print("- Organization: ", ", ".join([i for i in ent_org]))
        print("- Place: ", ", ".join([i for i in ent_loc]))
        print()

In [87]:
def print_events(df=df_nk, label=100):
    event = df[df.label == label]['title'].iloc[::-1].values.tolist()
    print("[ On-Issue Events ]\n")
    print(" -> ".join([i for i in event]))
    print()

In [ ]:
# A team mate of mine doesn't code at all. Thank you very much.

In [88]:
sents_nk = list(map(preprocess_manual, sents_nk))
print_events(df=df_nk, label=100)
extract_entity(sents=sents_nk, label=100, df=df_nk)

[ On-Issue Events ]

N. Korea's SLBM test conducted from barge, not from submarine: U.S. expert -> Pyongyang preparing to test launch tube used in SLBM: 38 North -> Pentagon declines comment on N.K.'s SLBM test -> North Korea successfully conducts SLBM test last month: U.S. report -> North Korea test-fired SLBM last month: South Korean military -> N. Korea believed to have conducted latest SLBM test from barge, not submarine -> N. Korea continues SLBM development: 38 North -> N.K. apparently fired ballistic missile from submarine: S. Korean military -> N. Korea aims to build new 3,000-ton sub armed with 3 SLBMs: experts -> Satellite imagery shows N. Korea actively pursuing SLBM development: 38 North -> N. Korea developing larger-class submarine for missile launch: expert -> New activity seen at test stand of N. Korea's SLBM development site: 38 North -> Satellite imagery shows progress in N. Korea's SLBM program

[ Detailed Information (per event) ]

Event:  Satellite imagery shows pro

In [89]:
sents_politics = list(map(preprocess_manual, sents_politics))
print_events(df=df_politics, label=1939)
extract_entity(sents=sents_politics, label=1939, df=df_politics)

[ On-Issue Events ]

Park seeks to upgrade relations in Gulf -> President Park begins trip to Middle East -> Park seeks to generate ‘second Middle East boom’ -> Korea, Kuwait agree to diversify business relations -> Park to meet with S. Korean businessmen in Qatar -> Park to hold 2nd summit with Qatari emir in four months -> Korea, Qatar to bolster economic partnership -> Park returns home after Mideast swing

[ Detailed Information (per event) ]

Event:  Park returns home after Mideast swing
- Person:  Hye Trip, Ju Chul Ki, Hye Sheikh, Mansour Bin Zayed Al Nahyan, United Arab, Cheong Wa Dae, Sabah Al Ahmad Al Jaber Al Sabah, Park, Salman Bin Abdulaziz Al Saud, Ju, Bin Talal, Bin Abdulaziz Al Saud, Mohammed Bin Zayed Bin Al Nahyan, Tamim Bin Hamad Al Thani
- Organization:  Gulf Nation, European African Study Institute Foreign Affair National, Hyundai Research Institute Korean Builder, Forecast Cambridge Energy Research, Price Dubai, Hri, Western Allied Gulf Nation, Crown Abu Dhabi, Cho